In [7]:
from os.path import join
from data import build_corpus, build_corpus_origin
from utils import load_model, extend_maps, prepocess_data_for_lstmcrf
from models.util import sort_by_lengths, tensorized

In [36]:
bilstm_model = load_model("E:/Project/KG/named_entity_recognition_bilstm/ckpts/bilstm.pkl")

In [3]:
bilstm_model

In [20]:
train_word_lists, train_tag_lists, word2id, tag2id = build_corpus_origin("train")
dev_word_lists, dev_tag_lists = build_corpus_origin("dev", make_vocab=False)
test_word_lists, test_tag_lists = build_corpus_origin("test", make_vocab=False)
bilstm_word2id, bilstm_tag2id = extend_maps(word2id, tag2id, for_crf=False)
# crf_word2id, crf_tag2id = extend_maps(word2id, tag2id, for_crf=True)
# train_word_lists, train_tag_lists = prepocess_data_for_lstmcrf(train_word_lists, train_tag_lists)
# dev_word_lists, dev_tag_lists = prepocess_data_for_lstmcrf(dev_word_lists, dev_tag_lists)
# test_word_lists, test_tag_lists = prepocess_data_for_lstmcrf(test_word_lists, test_tag_lists, test=True)

In [13]:
tag2id

# "B-NAME M-NAME E-NAME S-NAME 姓名"
# "B-CONT M_CONT E_CONT 国籍"
# "B-RACE M-RACE E-RACE S-RACE 民族"
# "B-TITLE M-TITLE E-TITLE 头衔"
# "B-EDU M-EDU E-EDU 学历"
# "B-ORG M-ORG E-ORG S-ORG 组织"
# "B-PRO M-PRO E-PRO 专业"
# "B-LOC M-LOC E-LOC 地点"

{'B-NAME': 0,
 'E-NAME': 1,
 'O': 2,
 'B-CONT': 3,
 'M-CONT': 4,
 'E-CONT': 5,
 'B-RACE': 6,
 'E-RACE': 7,
 'B-TITLE': 8,
 'M-TITLE': 9,
 'E-TITLE': 10,
 'B-EDU': 11,
 'M-EDU': 12,
 'E-EDU': 13,
 'B-ORG': 14,
 'M-ORG': 15,
 'E-ORG': 16,
 'M-NAME': 17,
 'B-PRO': 18,
 'M-PRO': 19,
 'E-PRO': 20,
 'S-RACE': 21,
 'S-NAME': 22,
 'B-LOC': 23,
 'M-LOC': 24,
 'E-LOC': 25,
 'M-RACE': 26,
 'S-ORG': 27}

In [5]:
word_lists, tag_lists, indices = sort_by_lengths(test_word_lists, test_tag_lists)

In [6]:
len(train_word_lists)

3821

In [16]:
def seperate_ch(sequence):
    return [ch for ch in sequence]
    # return [ch for ch in sequence]

In [17]:
test_word_lists[0]

['常', '建', '良', '，', '男', '，']

In [44]:
test = seperate_ch('什么鬼？"哈利说到，“我的金妮呢？”')
test

['什',
 '么',
 '鬼',
 '？',
 '"',
 '哈',
 '利',
 '说',
 '到',
 '，',
 '“',
 '我',
 '的',
 '金',
 '妮',
 '呢',
 '？',
 '”']

In [57]:
tensorized_sents, lengths = tensorized([train_word_lists[0]], bilstm_word2id)

In [58]:
tensorized_sents = tensorized_sents.cuda()

In [59]:
tensorized_sents.shape

torch.Size([1, 17])

In [60]:
batch_tagids = bilstm_model.model.test(tensorized_sents, lengths, bilstm_tag2id)

In [61]:
# 将id转化为标注
crf = True

pred_tag_lists = []
id2tag = dict((id_, tag) for tag, id_ in bilstm_tag2id.items())
for i, ids in enumerate(batch_tagids):
    tag_list = []

    if crf:
        for j in range(lengths[i] - 1):  # crf解码过程中，end被舍弃
            tag_list.append(id2tag[ids[j].item()])
    else:
        for j in range(lengths[i]):
            tag_list.append(id2tag[ids[j].item()])
    
    pred_tag_lists.append(tag_list)

In [62]:
pred_tag_lists

[['B-NAME',
  'E-NAME',
  'O',
  'O',
  'O',
  'B-CONT',
  'M-CONT',
  'M-CONT',
  'E-CONT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']]